# Setup

In [ ]:
import os
import sys
from pathlib import Path
import random
import numpy as np
import pickle
from typing import List

# Add the src directory to Python path
olmo_core_path = Path.cwd() / "src"
if olmo_core_path.exists():
    sys.path.insert(0, str(olmo_core_path))

from olmo_core.data import (
    NumpyDataLoaderConfig,
    NumpyDatasetConfig,
    NumpyDatasetType,
    TokenizerConfig,
)
from olmo_core.data.numpy_dataset import (
    VSLCurriculumType,
    VSLCurriculumConfig,
)


In [ ]:
# Set your new cache base directory (change this to your preferred location)
cache_base = "/home/joberant/NLP_2425b/yoavbaron"

# Set all relevant Hugging Face cache directories
os.environ["HF_HOME"] = cache_base
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_base, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_base, "datasets")
os.environ["HF_TOKENIZERS_CACHE"] = os.path.join(cache_base, "tokenizers")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from olmo_eval import HFTokenizer
from datasets import load_dataset

# Prepare the dataset and dataloader

In [ ]:
tokenizer_config = TokenizerConfig.dolma2()
tokenizer = HFTokenizer(
            tokenizer_config.identifier,
            pad_token_id=tokenizer_config.pad_token_id,
            eos_token_id=tokenizer_config.eos_token_id,
            bos_token_id=tokenizer_config.bos_token_id,
        )

include_instance_metadata = False # Set to true when you want tp retrieve metadata, during training set this to False
work_dir = "/home/joberant/NLP_2425b/shirab6/knowledge-analysis-suite/OLMo-core/hp_final/dataset-cache"

dataset_config = NumpyDatasetConfig.glob(
    "/home/morg/students/gottesman3/knowledge-analysis-suite/dolma/python/final_tokenizations_with_offsets/no_special/*.npy",  # can be globs
    name=NumpyDatasetType.kas_vsl,
    max_sequence_length=2048,
    min_sequence_length=64,
    vsl_curriculum=VSLCurriculumConfig(name=VSLCurriculumType.grow_p2, num_cycles=8, balanced=False),
    tokenizer=tokenizer_config,
    work_dir=str(work_dir),
    include_instance_metadata=include_instance_metadata,
)
kas_dataset = dataset_config.build()

In [ ]:
data_loader_config = NumpyDataLoaderConfig(
    global_batch_size=32768,
    seed=0,
    num_workers=8,
    prefetch_factor = 16,
)

dataloader = data_loader_config.build(kas_dataset)
dataloader.reshuffle(1)

# Load PopQA dataset and filter entities

In [ ]:
def get_important_chunks(dataset, min_num_chunks, max_num_chunks, instance_lengths):
    # Filter the dataset
    filtered_dataset = dataset['train'].filter(
        lambda example: min_num_chunks <= example['subject_num_chunks'] <= max_num_chunks
    )   

    # Create list of dictionaries with subject info and chunk lengths
    result_list = []    

    for example in filtered_dataset:
        
        subject_name = example['subj']
        subject_id = example['subj_id']
        chunks = example['subject_chunks']
        num_chunks = example['subject_num_chunks']

        chunk_lengths = instance_lengths[chunks]

        if subject_name == 'Madison':
            print(chunks)

        # Sort chunks by their lengths (descending order)
        if len(chunk_lengths) > 0:
            # Create pairs of (chunk, length) and sort by length
            chunk_length_pairs = list(zip(chunks, chunk_lengths))
            chunk_length_pairs.sort(key=lambda x: x[1], reverse=True)

            # Separate back into sorted chunks and lengths
            sorted_chunks = [pair[0] for pair in chunk_length_pairs]
            sorted_lengths = [pair[1] for pair in chunk_length_pairs]
        else:
            sorted_chunks = chunks
            sorted_lengths = chunk_lengths

        subject_dict = {
            'entity_id': subject_id,
            'entity_name': subject_name,
            'num_chunks': num_chunks,
            'chunks': sorted_chunks,
            'chunks_lengths': sorted_lengths
        }
        
        result_list.append(subject_dict)    

    # Sort the list by number of chunks (descending order)
    result_list.sort(key=lambda x: x['num_chunks'], reverse=True)

    return result_list


In [ ]:
ds = load_dataset("dhgottesman/popqa-kas")

"""
importsnt chunks has the following structure:
        {
            'entity_id': subject_id,
            'num_chunks': num_chunks,
            'chunks': sorted_chunks,
            'chunks_lengths': sorted_lengths
        }
"""
important_chunks = get_important_chunks(ds, 50, 100, kas_dataset.get_instance_lengths())

In [ ]:
def get_disjoint_entities(subject_dicts, seed = 406):
    used_chunks = set()
    disjoint_entities = []

    random.seed(seed)

    shuffled = subject_dicts.copy()
    random.shuffle(shuffled)

    for entity in shuffled:
        entity_chunks = set(entity['chunks'])

        # Check if entity has any overlapping chunk
        if used_chunks.isdisjoint(entity_chunks):
            disjoint_entities.append(entity)
            used_chunks.update(entity_chunks)

    return disjoint_entities


In [ ]:
max_seed = 0
max_len = 0
for i in range(5000):
    current_len = len(get_disjoint_entities(important_chunks, i))
    if current_len > max_len:
        max_len = current_len
        max_seed = i

max_seed, max_len

In [ ]:
disjoint_entities = get_disjoint_entities(important_chunks, max_seed)

In [ ]:
chunks_set = ()

for entity in disjoint_entities:
    for chunk in entity['chunks']:
        if chunk in chunks_set:
            print("not disjoint")
            break

# Load original batches

In [ ]:
all_batches = np.load("/home/morg/students/gottesman3/knowledge-analysis-suite/OLMo-core/batch_indices.npy", allow_pickle=True)

# Sample injection points

In [ ]:
import random

def sample_injection_points(total_steps, num_points_to_sample, max_num_chunks, interval, seed=None):
    """
    Samples unique injection points from a valid starting range to avoid overflow 
    when assigning chunk indices.

    Args:
        total_steps (int): The maximum possible step value (exclusive upper bound).
        num_points_to_sample (int): Number of injection points to sample.
        max_num_chunks (int): Maximum num_chunks across all entities.
        interval (int): Distance between chunk indices.
        seed (int, optional): Seed for reproducibility.

    Returns:
        List[int]: Sorted list of valid injection starting points.
    """
    if seed is not None:
        random.seed(seed)

    max_valid_start = total_steps - (max_num_chunks - 1) * interval
    if max_valid_start <= 0:
        raise ValueError("Interval and chunk size too large for total steps.")

    if num_points_to_sample > max_valid_start:
        raise ValueError("Cannot sample more injection points than available valid start points.")

    sampled_points = random.sample(range(max_valid_start), k=num_points_to_sample)
    return sorted(sampled_points)


def assign_indices_to_entities(entities, injection_points, interval):
    """
    Assigns indices to each entity starting at a given injection point with spacing.

    Args:
        entities (List[dict]): List of entity dicts.
        injection_points (List[int]): List of sampled injection start points.
        interval (int): Distance between chunk indices.

    Returns:
        Dict[str, List[int]]: Mapping from entity name to list of indices.
    """
    if len(entities) != len(injection_points):
        raise ValueError("Number of entities must match number of injection points.")

    result = {}

    for entity, start in zip(entities, injection_points):
        entity_id = entity['entity_id']
        num_chunks = entity['num_chunks']
        indices = [start + i * interval for i in range(num_chunks)]
        result[entity_id] = indices

    return result



In [ ]:
interval = 1

total_number_of_batches = dataloader.total_batches
injection_points = sample_injection_points(total_number_of_batches, len(disjoint_entities), 100, interval, 0)
all_injection_points_per_entity = assign_indices_to_entities(disjoint_entities, injection_points, interval)
# all_injection_points_per_entity

In [ ]:
import itertools
inj = sorted(itertools.chain.from_iterable(all_injection_points_per_entity.values()))[:10]
print(inj)


In [ ]:
for i, batch in enumerate(dataloader):
    if i == inj[0]:
        print(len(batch['index']), batch['index'])
        break
    if i > max(inj):
        break


# Build Swapping Dictionary

In [ ]:
import math

def shloop(
    injection_points: List[int],
    entity_data: dict,
    batch_to_chunks_map: dict,
    blacklist = []
) -> List[List]:
    """
    """
    # 1. Get entity chunks available for swapping and their lengths
    fixed_lengths = [2 ** math.ceil(math.log2(length)) for length in entity_data['chunks_lengths']]
    
    ent_chunk_to_len = dict(zip(entity_data['chunks'], fixed_lengths))
    ent_len_to_chunk = {v: k for k, v in ent_chunk_to_len.items()}

    # casting to int but might want to edit this
    batch_id_to_len = {}
    batch_len_to_id = {}
    for batch in injection_points:
        batch_len = int(32768 / len(batch_to_chunks_map[batch]))
        batch_id_to_len[batch] = batch_len
        batch_len_to_id[batch_len] = batch

    # 2. Calculate the injection span
    num_chunks = len(entity_data['chunks'])
    #print(f"Injection span: {list(injection_points)}")
    if len(injection_points) != num_chunks:
        f"Entity {entity_data['entity_id']} expected {num_chunks} injection points, but got {len(injection_points)}."
    

    sb = sorted(batch_len_to_id.keys())   
    se = sorted(ent_len_to_chunk.keys())
    #bucket_length = 2 ** math.ceil(math.log2(length))
    chunks_to_batches = []
    for len_e in se:
        for len_b in sb:
            if len_b == len_e:
                #print(len_e, len_b)
                chunk_id = ent_len_to_chunk[len_e]
                batch_id = batch_len_to_id[len_b]

                #print(f"Chunk {chunk_id} with length {len_e} will be swapped with batch {batch_id} with length {len_b}")
                # get a random chunk id from the batch
                chunk_id_from_batch = random.choice(batch_to_chunks_map[batch_id])
                while chunk_id_from_batch in blacklist:
                    chunk_id_from_batch = random.choice(batch_to_chunks_map[batch_id])

                if [chunk_id, chunk_id_from_batch] in chunks_to_batches or [chunk_id_from_batch, chunk_id] in chunks_to_batches:
                    print(chunk_id, chunk_id_from_batch, "already in")
                
                chunks_to_batches.append([chunk_id, chunk_id_from_batch])
                chunks_to_batches.append([chunk_id_from_batch, chunk_id])
                #chunks_to_batches[chunk_id] = chunk_id_from_batch # chunk e goes to chunk e' in batch b
                #chunks_to_batches[chunk_id_from_batch] = chunk_id # add the symetric mapping

                ent_len_to_chunk.pop(len_e) # pop one of the lengths
                ent_chunk_to_len.pop(chunk_id) # pop the chunk from the entity and pop one of the lengths
                batch_len_to_id.pop(len_b)
                batch_id_to_len.pop(batch_id) # pop the batch and the length from the batch
                break
                
    # ranmly match the rest of the chunks
    for chunk_id, batch_id in zip(ent_chunk_to_len.keys(), batch_id_to_len.keys()):
        if chunk_id not in chunks_to_batches:
            chunk_id_from_batch = random.choice(batch_to_chunks_map[batch_id])
            while chunk_id_from_batch in blacklist:
                    chunk_id_from_batch = random.choice(batch_to_chunks_map[batch_id])
                    
            if [chunk_id, chunk_id_from_batch] in chunks_to_batches or [chunk_id_from_batch, chunk_id] in chunks_to_batches:
                    print(chunk_id, chunk_id_from_batch, "already in")
                
            chunks_to_batches.append([chunk_id, chunk_id_from_batch])
            chunks_to_batches.append([chunk_id_from_batch, chunk_id])
            
            #chunks_to_batches[chunk_id] = chunk_id_from_batch
            #chunks_to_batches[chunk_id_from_batch] = chunk_id

    return chunks_to_batches

In [ ]:
full_mapping = []
blacklist = set()
for entity in disjoint_entities:
    blacklist.update(entity['chunks'])

for i, important_chunk in enumerate(disjoint_entities):
    pts = all_injection_points_per_entity[important_chunk['entity_id']]

    # The 'important_chunk' variable is the integer you need.
    # Pass it directly to your function.
    res = shloop(
        pts,
        important_chunk,
        all_batches,
        blacklist
    )
    
    # add the chunks already in the injection mapping to a blacklist so they don't get sampled
    blacklist.update(list(set([r[0] for r in res])))

    # extend full mapping with the result
    full_mapping.extend(res)


In [ ]:
normal_dict = {}
for key, value in full_mapping:
    normal_dict[key] = value

In [ ]:
len(normal_dict), len(full_mapping)

In [ ]:
total_chunks = sum(entity['num_chunks'] for entity in disjoint_entities)
print("Combined amount of chunks in disjoint entities:", total_chunks)

# Statistics

In [ ]:
from collections import defaultdict

# Track chunk_id -> list of entity_ids where it appears
chunk_to_entities = defaultdict(set)

for entity in disjoint_entities:
    entity_id = entity['entity_id']
    for chunk_id in entity['chunks']:
        chunk_to_entities[chunk_id].add(entity_id)

# Find chunk_ids that appear in more than one entity
duplicate_chunks = {chunk_id: list(entity_ids) for chunk_id, entity_ids in chunk_to_entities.items() if len(entity_ids) > 1}

print(f"Number of chunk ids appearing in multiple entities: {len(duplicate_chunks)}")
for chunk_id, entity_ids in list(duplicate_chunks.items())[:-1]:  # show first 10 for brevity
    print(f"Chunk ID {chunk_id} appears in entities: {entity_ids}")

In [ ]:
import itertools
shared_counts = defaultdict(lambda: defaultdict(int))

for chunk_id, entities in chunk_to_entities.items():
        # A chunk must be in at least two entities to be "shared".
        if len(entities) < 2:
            continue

        # 2. Generate all unique pairs of entities for the current chunk.
        # For a list [A, B, C], this yields (A, B), (A, C), (B, C).
        for entity1, entity2 in itertools.combinations(entities, 2):
            # 3. Increment the count for both directions of the pair.
            shared_counts[entity1][entity2] += 1
            shared_counts[entity2][entity1] += 1

In [ ]:
opp = {}
# Sort entities by ID for consistent output order.
for entity_id in sorted(shared_counts.keys()):
    # Get the dictionary of shared counts for the current entity.
    connections = shared_counts[entity_id]
    
    # Sort the connections by count (value) in descending order.
    sorted_connections = dict(sorted(
        connections.items(),
        key=lambda item: item[1],
        reverse=True
    ))
    opp[entity_id] = sorted_connections

In [ ]:
import matplotlib.pyplot as plt

# Prepare data for plotting: top N entities with most connections
N = 20
entity_ids = list(opp.keys())
num_connections = [len(opp[eid]) for eid in entity_ids]

In [ ]:
important_chunk_4026 = next((chunk for chunk in important_chunks if chunk['entity_id'] == 4026), None)
print(important_chunk_4026)

In [ ]:
from collections import Counter

import matplotlib.pyplot as plt

# Count how many entities have each number of connections
connection_counts = Counter(num_connections)

# Prepare data for plotting
x = sorted(connection_counts.keys())
y = [connection_counts[k] for k in x]

plt.figure(figsize=(10, 6))
plt.bar(x, y)
plt.xlabel("Number of Connections")
plt.ylabel("Number of Entities")
plt.title("Distribution of Entity Connections")
plt.tight_layout()
plt.show()

In [ ]:
# Sort and select top N
sorted_indices = sorted(range(len(num_connections)), key=lambda i: num_connections[i], reverse=True)[:N]
top_entity_ids = [entity_ids[i] for i in sorted_indices]
top_num_connections = [num_connections[i] for i in sorted_indices]

In [ ]:
import networkx as nx

import matplotlib.pyplot as plt

# Create a graph
G = nx.Graph()

# Add nodes (entities)
G.add_nodes_from(opp.keys())

# Add edges with weights (number of shared chunks)
for entity_id, connections in opp.items():
    for other_id, weight in connections.items():
        if weight > 1:
            G.add_edge(entity_id, other_id, weight=weight)

# Draw the graph
plt.figure(figsize=(16, 12))
pos = nx.spring_layout(G, k=0.15, seed=42)
edges = G.edges()
weights = [G[u][v]['weight'] for u, v in edges]

nx.draw_networkx_nodes(G, pos, node_size=100, node_color='skyblue')
nx.draw_networkx_edges(G, pos, edgelist=edges, width=[w/2 for w in weights], alpha=0.7)
#nx.draw_networkx_labels(G, pos, font_size=8)
# Remove node labels for a cleaner graph
# (Do not call nx.draw_networkx_labels)
plt.title("Entity Connection Graph (Edge weight = shared chunk count)")
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
from collections import Counter

import matplotlib.pyplot as plt

# Count how many entities each chunk appears in
chunk_entity_counts = [len(entity_ids) for entity_ids in duplicate_chunks.values()]

# Count frequency of each "number of entities"
chunk_count_freq = Counter(chunk_entity_counts)

# Prepare data for plotting
x_chunk = sorted(chunk_count_freq.keys())
y_chunk = [chunk_count_freq[k] for k in x_chunk]

plt.figure(figsize=(10, 6))
plt.bar(x_chunk, y_chunk)
plt.xlabel("Number of Entities a Chunk Appears In")
plt.ylabel("Number of Chunks")
plt.title("Distribution of Chunk Occurrences Across Entities")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.bar(range(len(top_entity_ids)), top_num_connections, tick_label=top_entity_ids)
plt.xlabel("Entity ID")
plt.ylabel("Number of Connections")
plt.title("Top Entities by Number of Connections")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
average_connections = np.mean(num_connections)
print("Average number of connections per entity:", average_connections)

In [ ]:
median_connections = np.median(num_connections)
print("Median number of connections per entity:", median_connections)

In [ ]:
num_entities_gt_10 = sum(1 for connections in opp.values() if len(connections) > 10)
print("Number of entities sharing chunks with more than 10 connections:", num_entities_gt_10)

In [ ]:
total_len = 0
for d in full_mapping:
    total_len += len(d)

total_len / 2

# Save and load the swapping dict

In [ ]:
with open('/home/joberant/NLP_2425b/yoavbaron/knowledge-analysis-suite/OLMo-core/swapping_dict.pkl', 'wb') as f:
    pickle.dump(normal_dict, f)

In [ ]:
# To load it back later
with open('/home/joberant/NLP_2425b/yoavbaron/knowledge-analysis-suite/OLMo-core/swapping_dict.pkl', 'rb') as f:
    swapping_dict = pickle.load(f)

# Rebuild dataset and dataloader with swapped chunk indices

In [ ]:
tokenizer_config = TokenizerConfig.dolma2()
tokenizer = HFTokenizer(
            tokenizer_config.identifier,
            pad_token_id=tokenizer_config.pad_token_id,
            eos_token_id=tokenizer_config.eos_token_id,
            bos_token_id=tokenizer_config.bos_token_id,
        )

include_instance_metadata = False # Set to true when you want tp retrieve metadata, during training set this to False
work_dir = "/home/joberant/NLP_2425b/shirab6/knowledge-analysis-suite/OLMo-core/hp_final/dataset-cache"


dataset_config = NumpyDatasetConfig.glob(
    "/home/morg/students/gottesman3/knowledge-analysis-suite/dolma/python/final_tokenizations_with_offsets/no_special/*.npy",  # can be globs
    name=NumpyDatasetType.kas_vsl,
    max_sequence_length=2048,
    min_sequence_length=64,
    vsl_curriculum=VSLCurriculumConfig(name=VSLCurriculumType.grow_p2, num_cycles=8, balanced=False),
    tokenizer=tokenizer_config,
    work_dir=str(work_dir),
    include_instance_metadata=include_instance_metadata,
    swapping_dict = swapping_dict,
)

reordered_dataset = dataset_config.build()


In [ ]:
data_loader_config = NumpyDataLoaderConfig(
    global_batch_size=32768,
    seed=0,
    num_workers=8,
    prefetch_factor = 16,
)

dataloader = data_loader_config.build(reordered_dataset)
dataloader.reshuffle(1)

In [ ]:
missing_chunks = []
for entity in disjoint_entities:
    for chunk_id in entity['chunks']:
        if chunk_id not in swapping_dict:
            missing_chunks.append(chunk_id)

print(f"Number of missing important chunks: {len(missing_chunks)}")
if missing_chunks:
    print("Missing chunk IDs:", missing_chunks[:20])  # show first 20 for brevity
else:
    print("All important chunks are present in swapping_dict.")

In [ ]:
sorted_keys = list(swapping_dict.keys())
sorted_keys.sort()

In [ ]:
for i, batch in enumerate(dataloader):
    if i == inj[0]:
        print(len(batch['index']), batch['index'])
        break
    if i > max(inj):
        break

In [ ]:
orig = [ 7631809,  6940641,  3542626,  9931434,  9470964,  5211576,  4257442,
          109997, 10159240,  2756754,  8779079,  2222801,  4308700,  1495528,
         2898556,  2697159, 10461777,   136076,  5838211,  7914579,  3234602,
         2059624,  9026918,  6059905,  7357863,  3345625,   818789,  4348662,
         2368892,  9938633,   264110, 10054712,  1550785,  2556067,   235054,
         7588519,  1821616,  8467303,  5276389,   273819,  6845202,  8054962,
           45173,  1475641, 10126768,  6707865,  8668042,  9398828,  2105198,
         5300031,  7318456,  3294546,  8825413,  5412833,   362726, 10035273,
         7699557,  7837840,  8854210,  9661840,  6383184,   206394,  6719335,
        10423236,  8018408,  5597199,  5276516,  5231316,  2196400, 10267688,
         1521057,  4652225,  9679001,  9652065,  9429994,  2282572,  6956468,
         8611192,  7721421,  6933702,  4336640,  6175126,  8009534,  3572447,
         5267884,  1575983,  3548330,  7403549,  6448858,  1431713,   827523,
         6320696,  3430047,  5381881,  9632193,  9383363,  1606688, 10108639,
         3705895, 10258338,  9717646,  6062364,   482440,  7780681,  1045536,
         8046308,  2504279,  3943587,  2441651,  3855765,  4084704,  6087419,
         4892372,  5649860,  6118288,   750845,  8505029,  9260357,  9635432,
         4270765, 10089093, 10223830,  3740823,  8578604,  3763499,  9205088,
         9813411,  6419315,  6763331,  9119661,  9088378,  5766675,  9326163,
         7975639,  1991544,   611044,  3355656,  2055100,  9704132,  6231878,
         2614424,  2053407,  8520540,   883889,  2137193,  7605850,    57965,
         6222716,  3650282,  5265742,  4984812, 10008793,  8164404,  5001298,
         4981525,  5767203,   695583,  6776778,  9820583,   711651,  4299634,
          593114,  5222237,  9881415,  7831993,  4950886,  5730604,  2090383,
         8206375,  6587449,  2826392,  4036001,  4466790,  5539846,  9809955,
         5364018,  3159089,  4362151,  5900454,  6523105,  8326342,  8936973,
         6565883,  8906095,  8984150,   541885,  4060055,  2743755,  7070246,
         9201059, 10430806,  1613698,  1398794,  6782600,  5851072,  9825166,
         7021826,  1494532,  8008803,  2901043,  9942393,  8044859,    35542,
         1997732,  9884157,  7442658, 10166251,  5547095,   561454,  2844523,
           25535,  7805619,  6612396,   606006,  2505521,  8799493,  1906972,
         7157075,  6700643,  4498508,  9373815,  5539025,  2517344,  5399511,
         5084722,  2777578,  5642244,  9995602,  9493457,  2310500,  4885621,
         2002396,  5051772,  9958852,  3151452,  4636379,  6961252,  5910843,
         6056337,  6797837,   739182,  6538852,  5331249,  5909444,  5970866,
           73284,  6716057,  7959418,  4999099,  6817239,  8087901,  6826491,
         5403531,  1104048,  4690218,  3440652]

In [ ]:
swp = [[ 5303980,  3401112,  2619408,  7631809,  6940641,  3542626,  9931434,
         9470964,  5211576,  4257442,   109997, 10159240,  2756754,  8779079,
         2222801,  4308700,  1495528,  2898556,  2697159, 10461777,   136076,
         5838211,  7914579,  3234602,  2059624,  9026918,  6059905,  7357863,
         3345625,   818789,  4348662,  2368892,  9938633,   264110, 10054712,
         1550785,  2556067,   235054,  7588519,  1821616,  8467303,  5276389,
          273819,  6845202,  8054962,    45173,  1475641, 10126768,  6707865,
         8668042,  9398828,  2105198,  5300031,  7318456,  3294546,  8825413,
         5412833,   362726, 10035273,  7699557,  7837840,  8854210,  9661840,
         6383184,   206394,  6719335, 10423236,  8018408,  5597199,  5276516,
         5231316,  2196400, 10267688,  1521057,  4652225,  9679001,  9652065,
         9429994,  2282572,  6956468,  8611192,  7721421,  6933702,  4336640,
         6175126,  8009534,  3572447,  5267884,  1575983,  3548330,  7403549,
         6448858,  1431713,   827523,  6320696,  3430047,  5381881,  9632193,
         9383363,  1606688, 10108639,  3705895, 10258338,  9717646,  6062364,
          482440,  7780681,  1045536,  8046308,  2504279,  3943587,  2441651,
         3855765,  4084704,  6087419,  4892372,  5649860,  6118288,   750845,
         8505029,  9260357,  9635432,  4270765, 10089093, 10223830,  3740823,
         8578604,  3763499,  9205088,  9813411,  6419315,  6763331,  9119661,
         9088378,  5766675,  9326163,  7975639,  1991544,   611044,  3355656,
         2055100,  9704132,  6231878,  2614424,  2053407,  8520540,   883889,
         2137193,  7605850,    57965,  6222716,  3650282,  5265742,  4984812,
        10008793,  8164404,  5001298,  4981525,  5767203,   695583,  6776778,
         9820583,   711651,  4299634,   593114,  5222237,  9881415,  7831993,
         4950886,  5730604,  2090383,  8206375,  6587449,  2826392,  4036001,
         4466790,  5539846,  9809955,  5364018,  3159089,  4362151,  5900454,
         6523105,  8326342,  8936973,  6565883,  8906095,  8984150,   541885,
         4060055,  2743755,  7070246,  9201059, 10430806,  1613698,  1398794,
         6782600,  5851072,  9825166,  7021826,  1494532,  8008803,  2901043,
         9942393,  8044859,    35542,  1997732,  9884157,  7442658, 10166251,
         5547095,   561454,  2844523,    25535,  7805619,  6612396,   606006,
         2505521,  8799493,  1906972,  7157075,  6700643,  4498508,  9373815,
         5539025,  2517344,  5399511,  5084722,  2777578,  5642244,  9995602,
         9493457,  2310500,  4885621,  2002396,  5051772,  9958852,  3151452]]

In [ ]:
# Flatten swp if it's a list of lists
swp_flat = swp[0] if isinstance(swp[0], (list, np.ndarray)) else swp

# Find elements in orig not in swp_flat
orig_not_in_swp = [x for x in orig if x not in swp_flat]
# Find elements in swp_flat not in orig
swp_not_in_orig = [x for x in swp_flat if x not in orig]

print("In orig but not in swp:", orig_not_in_swp)
print("In swp but not in orig:", swp_not_in_orig)